In [47]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import sys 
import os
import shapely
from shapely import Point, Polygon
sys.path.append('../src/')

from data_utils import *

In [48]:
# read ground truth detection labels

gt_path = '/home/brwei01/Data/data_label/training/label_02/0000.txt'
col_names = ['frame', 'track_id', 'type', 'truncated', 'occluded', 'alpha', 'bbox_left', 'bbox_top', 
             'bbox_right', 'bbox_bottom', 'height', 'width', 'length', 'pos_x', 'pos_y', 'pos_z', 'rot_y']
df_gt = pd.read_csv(gt_path, header=None, sep=' ')
df_gt.columns = col_names
df_gt.head()

,frame,track_id,type,truncated,occluded,alpha,bbox_left,bbox_top,bbox_right,bbox_bottom,height,width,length,pos_x,pos_y,pos_z,rot_y
0,0,-1,DontCare,-1,-1,-10.000000,219.310000,188.490000,245.500000,218.560000,-1000.000000,-1000.000000,-1000.000000,-10.000000,-1.000000,-1.000000,-1.000000
1,0,-1,DontCare,-1,-1,-10.000000,47.560000,195.280000,115.480000,221.480000,-1000.000000,-1000.000000,-1000.000000,-10.000000,-1.000000,-1.000000,-1.000000
2,0,0,Van,0,0,-1.793451,296.744956,161.752147,455.226042,292.372804,2.000000,1.823255,4.433886,-4.552284,1.858523,13.410495,-2.115488
3,0,1,Cyclist,0,0,-1.936993,737.619499,161.531951,931.112229,374.000000,1.739063,0.824591,1.785241,1.640400,1.675660,5.776261,-1.675458
4,0,2,Pedestrian,0,0,-2.523309,1106.137292,166.576807,1204.470628,323.876144,1.714062,0.767881,0.972283,6.301919,1.652419,8.455685,-1.900245


In [49]:
frame = 3
df_tracking_frame_gt = df_gt[df_gt.frame==frame]
boxes_gt = df_tracking_frame_gt[['bbox_left','bbox_top','bbox_right','bbox_bottom',]]
boxes_gt

,bbox_left,bbox_top,bbox_right,bbox_bottom
15,243.860000,168.790000,275.27000,204.270000
16,62.325000,165.380000,166.91000,201.500000
17,291.328036,145.082168,446.40083,269.811515
18,759.786603,146.098339,954.28016,374.000000
19,1154.836779,148.360923,1241.00000,321.627088


In [50]:
boxes_2d = np.array(boxes_gt)
boxes_2d

array([[ 243.86    ,  168.79    ,  275.27    ,  204.27    ],
       [  62.325   ,  165.38    ,  166.91    ,  201.5     ],
       [ 291.328036,  145.082168,  446.40083 ,  269.811515],
       [ 759.786603,  146.098339,  954.28016 ,  374.      ],
       [1154.836779,  148.360923, 1241.      ,  321.627088]])

In [51]:
boxes_gt_geom = []
for i in range(len(boxes_gt)):
    box_gt = boxes_gt.iloc[i]
    pt1 = [box_gt[0], box_gt[1]]
    pt2 = [box_gt[2], box_gt[1]]
    pt3 = [box_gt[2], box_gt[3]]
    pt4 = [box_gt[0], box_gt[3]] 
    box_gt = Polygon([pt1, pt2, pt3, pt4, pt1])
    boxes_gt_geom.append(box_gt)
    print(box_gt)

POLYGON ((243.86 168.79, 275.27 168.79, 275.27 204.27, 243.86 204.27, 243.86 168.79))
POLYGON ((62.325 165.38, 166.91 165.38, 166.91 201.5, 62.325 201.5, 62.325 165.38))
POLYGON ((291.328036 145.082168, 446.40083 145.082168, 446.40083 269.811515, 291.328036 269.811515, 291.328036 145.082168))
POLYGON ((759.786603 146.098339, 954.28016 146.098339, 954.28016 374, 759.786603 374, 759.786603 146.098339))
POLYGON ((1154.836779 148.360923, 1241 148.360923, 1241 321.627088, 1154.836779 321.627088, 1154.836779 148.360923))


## Read detection results by yolo-slam

In [53]:
# read detection results by yolo-slam

res_path = '/home/brwei01/Dev/COMP0130_22-23_Topic_03/Coursework_03/Results/console_log_formatted.txt'
col_names = ['frame', 'bbox_left', 'bbox_top', 'bbox_right', 'bbox_bottom', 'distance']
df_res = pd.read_csv(res_path, header=None, sep=' ')
df_res.columns = col_names
df_res.head()

,frame,bbox_left,bbox_top,bbox_right,bbox_bottom,distance
0,15,466,199,516,238,4.88335
1,15,224,187,260,211,4.88335
2,15,453,165,512,206,4.88335
3,15,240,188,271,209,4.88335
4,15,183,188,231,214,4.88335


In [54]:
frame = 18
df_tracking_frame_res = df_res[df_res.frame==frame]
boxes_res = df_tracking_frame_res[['bbox_left','bbox_top','bbox_right','bbox_bottom']]
boxes_res

,bbox_left,bbox_top,bbox_right,bbox_bottom
28,487,200,538,239
29,255,186,291,208
30,103,192,156,216
31,126,187,156,211
32,475,166,535,204
33,205,184,249,213
34,722,190,775,233
35,783,246,887,368
36,1021,184,1150,258


In [27]:
boxes_res_geom = []
for i in range(len(boxes_res)):
    box_res = boxes_res.iloc[i]
    pt1 = [box_res[0], box_res[1]]
    pt2 = [box_res[2], box_res[1]]
    pt3 = [box_res[2], box_res[3]]
    pt4 = [box_res[0], box_res[3]] 
    box_res = Polygon([pt1, pt2, pt3, pt4, pt1])
    boxes_res_geom.append(box_res)
    print(box_res)

POLYGON ((487 200, 538 200, 538 239, 487 239, 487 200))
POLYGON ((255 186, 291 186, 291 208, 255 208, 255 186))
POLYGON ((103 192, 156 192, 156 216, 103 216, 103 192))
POLYGON ((126 187, 156 187, 156 211, 126 211, 126 187))
POLYGON ((475 166, 535 166, 535 204, 475 204, 475 166))
POLYGON ((205 184, 249 184, 249 213, 205 213, 205 184))
POLYGON ((722 190, 775 190, 775 233, 722 233, 722 190))
POLYGON ((783 246, 887 246, 887 368, 783 368, 783 246))
POLYGON ((1021 184, 1150 184, 1150 258, 1021 258, 1021 184))
POLYGON ((126 197, 163 197, 163 220, 126 220, 126 197))
POLYGON ((5 204, 45 204, 45 226, 5 226, 5 204))
POLYGON ((480 202, 544 202, 544 247, 480 247, 480 202))
POLYGON ((204 191, 255 191, 255 220, 204 220, 204 191))
POLYGON ((482 172, 544 172, 544 210, 482 210, 482 172))
POLYGON ((731 198, 784 198, 784 239, 731 239, 731 198))
POLYGON ((774 248, 883 248, 883 371, 774 371, 774 248))
POLYGON ((1045 192, 1180 192, 1180 269, 1045 269, 1045 192))


## judge if the polygons are the same

In [28]:
selected_boxes_res = []
selected_boxes_gt = []
for box_gt in boxes_gt_geom:
    for box_res in boxes_res_geom:
        if box_res.contains(box_gt) or box_res.intersects(box_gt) or box_res.within(box_gt):
            selected_boxes_gt.append(box_gt)
            selected_boxes_res.append(box_res)


In [29]:
selected_boxes_res

[<POLYGON ((255 186, 291 186, 291 208, 255 208, 255 186))>,
 <POLYGON ((205 184, 249 184, 249 213, 205 213, 205 184))>,
 <POLYGON ((204 191, 255 191, 255 220, 204 220, 204 191))>,
 <POLYGON ((103 192, 156 192, 156 216, 103 216, 103 192))>,
 <POLYGON ((126 187, 156 187, 156 211, 126 211, 126 187))>,
 <POLYGON ((126 197, 163 197, 163 220, 126 220, 126 197))>,
 <POLYGON ((722 190, 775 190, 775 233, 722 233, 722 190))>,
 <POLYGON ((783 246, 887 246, 887 368, 783 368, 783 246))>,
 <POLYGON ((731 198, 784 198, 784 239, 731 239, 731 198))>,
 <POLYGON ((774 248, 883 248, 883 371, 774 371, 774 248))>,
 <POLYGON ((1045 192, 1180 192, 1180 269, 1045 269, 1045 192))>]

In [30]:
selected_boxes_gt

[<POLYGON ((243.86 168.79, 275.27 168.79, 275.27 204.27, 243.86 204.27, 243.8...>,
 <POLYGON ((243.86 168.79, 275.27 168.79, 275.27 204.27, 243.86 204.27, 243.8...>,
 <POLYGON ((243.86 168.79, 275.27 168.79, 275.27 204.27, 243.86 204.27, 243.8...>,
 <POLYGON ((62.325 165.38, 166.91 165.38, 166.91 201.5, 62.325 201.5, 62.325 ...>,
 <POLYGON ((62.325 165.38, 166.91 165.38, 166.91 201.5, 62.325 201.5, 62.325 ...>,
 <POLYGON ((62.325 165.38, 166.91 165.38, 166.91 201.5, 62.325 201.5, 62.325 ...>,
 <POLYGON ((759.787 146.098, 954.28 146.098, 954.28 374, 759.787 374, 759.787...>,
 <POLYGON ((759.787 146.098, 954.28 146.098, 954.28 374, 759.787 374, 759.787...>,
 <POLYGON ((759.787 146.098, 954.28 146.098, 954.28 374, 759.787 374, 759.787...>,
 <POLYGON ((759.787 146.098, 954.28 146.098, 954.28 374, 759.787 374, 759.787...>,
 <POLYGON ((1154.837 148.361, 1241 148.361, 1241 321.627, 1154.837 321.627, 1...>]

In [46]:
# Convert coordinates to left, top, right, bottom format
corners_coords = []
for polygon in selected_boxes_res:
    exterior_coords = np.array(polygon.exterior.coords)
    left = np.min(exterior_coords[:, 0])
    right = np.max(exterior_coords[:, 0])
    top = np.max(exterior_coords[:, 1])
    bottom = np.min(exterior_coords[:, 1])
    coords = [left, top, right, bottom]
    corners_coords.append(coords)
corners_coords = np.array(corners_coords)
corners_coords

array([[ 255.,  208.,  291.,  186.],
       [ 205.,  213.,  249.,  184.],
       [ 204.,  220.,  255.,  191.],
       [ 103.,  216.,  156.,  192.],
       [ 126.,  211.,  156.,  187.],
       [ 126.,  220.,  163.,  197.],
       [ 722.,  233.,  775.,  190.],
       [ 783.,  368.,  887.,  246.],
       [ 731.,  239.,  784.,  198.],
       [ 774.,  371.,  883.,  248.],
       [1045.,  269., 1180.,  192.]])